# 課題1：ドキュメントの類似度を計算する 
以下の要件を満たすプログラムを作成  
・10個以上の文書を対象  
・2種類以上の文書表現手法  
・2種類以上の距離尺度  

In [26]:
import re
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

## 対象となる10個以上の文書を取得
著作権の切れた青空文庫から１０個の文書を取得  
doc1 : 夏目漱石「こころ」上の一  
doc2 : 夏目漱石「こころ」上の二  
doc3 : 夏目漱石「こころ」上の三  
doc4 : 宮沢賢治「銀河鉄道の夜」一  
doc5 : 宮沢賢治「銀河鉄道の夜」二  
doc6 : 宮沢賢治「銀河鉄道の夜」三  
doc7 : 夏目漱石「坊ちゃん」一(途中まで)   
doc8 : 夏目漱石「坊ちゃん」二(途中まで)  
doc9 : 夏目漱石「坊ちゃん」三(途中まで)  
doc10 : フランツカフカ「変身」一(途中まで)

In [54]:
# データ読み込み
doc_origin_list = []
for i in range(1,11):
    with open("./doc_dir/doc{}.txt".format(i)) as f:
        s = f.read()
        doc_origin_list.append(s)

print(doc_origin_list[0])

　私《わたくし》はその人を常に先生と呼んでいた。だからここでもただ先生と書くだけで本名は打ち明けない。これは世間を憚《はば》かる遠慮というよりも、その方が私にとって自然だからである。私はその人の記憶を呼び起すごとに、すぐ「先生」といいたくなる。筆を執《と》っても心持は同じ事である。よそよそしい頭文字《かしらもじ》などはとても使う気にならない。
　私が先生と知り合いになったのは鎌倉《かまくら》である。その時私はまだ若々しい書生であった。暑中休暇を利用して海水浴に行った友達からぜひ来いという端書《はがき》を受け取ったので、私は多少の金を工面《くめん》して、出掛ける事にした。私は金の工面に二《に》、三日《さんち》を費やした。ところが私が鎌倉に着いて三日と経《た》たないうちに、私を呼び寄せた友達は、急に国元から帰れという電報を受け取った。電報には母が病気だからと断ってあったけれども友達はそれを信じなかった。友達はかねてから国元にいる親たちに勧《すす》まない結婚を強《し》いられていた。彼は現代の習慣からいうと結婚するにはあまり年が若過ぎた。それに肝心《かんじん》の当人が気に入らなかった。それで夏休みに当然帰るべきところを、わざと避けて東京の近くで遊んでいたのである。彼は電報を私に見せてどうしようと相談をした。私にはどうしていいか分らなかった。けれども実際彼の母が病気であるとすれば彼は固《もと》より帰るべきはずであった。それで彼はとうとう帰る事になった。せっかく来た私は一人取り残された。
　学校の授業が始まるにはまだ大分《だいぶ》日数《ひかず》があるので鎌倉におってもよし、帰ってもよいという境遇にいた私は、当分元の宿に留《と》まる覚悟をした。友達は中国のある資産家の息子《むすこ》で金に不自由のない男であったけれども、学校が学校なのと年が年なので、生活の程度は私とそう変りもしなかった。したがって一人《ひとり》ぼっちになった私は別に恰好《かっこう》な宿を探す面倒ももたなかったのである。
　宿は鎌倉でも辺鄙《へんぴ》な方角にあった。玉突《たまつ》きだのアイスクリームだのというハイカラなものには長い畷《なわて》を一つ越さなければ手が届かなかった。車で行っても二十銭は取られた。けれども個人の別荘はそこここにいくつでも建てられていた。それに海へはごく近いので海水浴をやるには至極便利な地位を

In [55]:
def cleaning_text(text):
    # 改行コードの削除
    text = re.sub(r"[\r\n]", '', text)    
    # 空欄の削除
    text = re.sub(r"[\u3000 \t]", '', text)    
    # 《》内を削除
    text = re.sub("《[^》]+》", '', text)
    # 句読点などの記号を削除
    text = re.sub("[。、―「」!?｜［］＃]", '', text)
    
    return text
 
doc_list = []
for doc in doc_origin_list:
    new_doc = cleaning_text(doc)
    doc_list.append(new_doc)

print(doc_list[0])

私はその人を常に先生と呼んでいただからここでもただ先生と書くだけで本名は打ち明けないこれは世間を憚かる遠慮というよりもその方が私にとって自然だからである私はその人の記憶を呼び起すごとにすぐ先生といいたくなる筆を執っても心持は同じ事であるよそよそしい頭文字などはとても使う気にならない私が先生と知り合いになったのは鎌倉であるその時私はまだ若々しい書生であった暑中休暇を利用して海水浴に行った友達からぜひ来いという端書を受け取ったので私は多少の金を工面して出掛ける事にした私は金の工面に二三日を費やしたところが私が鎌倉に着いて三日と経たないうちに私を呼び寄せた友達は急に国元から帰れという電報を受け取った電報には母が病気だからと断ってあったけれども友達はそれを信じなかった友達はかねてから国元にいる親たちに勧まない結婚を強いられていた彼は現代の習慣からいうと結婚するにはあまり年が若過ぎたそれに肝心の当人が気に入らなかったそれで夏休みに当然帰るべきところをわざと避けて東京の近くで遊んでいたのである彼は電報を私に見せてどうしようと相談をした私にはどうしていいか分らなかったけれども実際彼の母が病気であるとすれば彼は固より帰るべきはずであったそれで彼はとうとう帰る事になったせっかく来た私は一人取り残された学校の授業が始まるにはまだ大分日数があるので鎌倉におってもよし帰ってもよいという境遇にいた私は当分元の宿に留まる覚悟をした友達は中国のある資産家の息子で金に不自由のない男であったけれども学校が学校なのと年が年なので生活の程度は私とそう変りもしなかったしたがって一人ぼっちになった私は別に恰好な宿を探す面倒ももたなかったのである宿は鎌倉でも辺鄙な方角にあった玉突きだのアイスクリームだのというハイカラなものには長い畷を一つ越さなければ手が届かなかった車で行っても二十銭は取られたけれども個人の別荘はそこここにいくつでも建てられていたそれに海へはごく近いので海水浴をやるには至極便利な地位を占めていた私は毎日海へはいりに出掛けた古い燻ぶり返った藁葺の間を通り抜けて磯へ下りるとこの辺にこれほどの都会人種が住んでいるかと思うほど避暑に来た男や女で砂の上が動いていたある時は海の中が銭湯のように黒い頭でごちゃごちゃしている事もあったその中に知った人を一人ももたない私もこういう賑やかな景色の中に裹まれて砂の上

In [62]:
# 形態素解析
from janome.tokenizer import Tokenizer

t = Tokenizer()

doc_token = []
for doc in doc_list:
    doc_token.append(t.tokenize(doc, wakati=True))

## 2種類以上の文書表現手法
・集合ベース  
文書を単語の集合として表現  
・ベクトルベース  
文書をベクトルとして表現  

## 集合ベース

In [63]:
# 形態素解析した文書を単語の集合に変換
doc_set_list = []
for i in range(10):
    doc_set_list.append(set(doc_token[i]))

### 集合ベースの類似度
Jaccard係数  
Dice係数  
Simpson係数  

#### jaccard

In [65]:
from nltk.metrics import jaccard_distance
import itertools

cols = ['文書のペア', 'jaccard値']
ans_jaccard_df = pd.DataFrame(index=[], columns=cols)
#Jaccard係数を計算
for i in range(10):
    for j in range(10):
        if i < j:
            tmp_name = "doc" + str(i+1) + " と doc" +  str(j+1)
            record = pd.Series([tmp_name, 1 - jaccard_distance(doc_set_list[i], doc_set_list[j])], index=ans_jaccard_df.columns)
            ans_jaccard_df = ans_jaccard_df.append(record, ignore_index=True)

In [66]:
# ５個表示
ans_jaccard_df[1:5]

,文書のペア,jaccard値
1,doc1 と doc3,0.180505
2,doc1 と doc4,0.119792
3,doc1 と doc5,0.101562
4,doc1 と doc6,0.119134


In [67]:
# jaccard値を大きい順にソート
print(ans_jaccard_df.sort_values(by="jaccard値",ascending=False)[1:10])

          文書のペア  jaccard値
1   doc1 と doc3  0.180505
30  doc5 と doc6  0.174946
25  doc4 と doc6  0.170412
42  doc8 と doc9  0.166998
40  doc7 と doc9  0.161716
0   doc1 と doc2  0.157993
17  doc3 と doc4  0.156140
39  doc7 と doc8  0.155786
24  doc4 と doc5  0.154158


In [68]:
# jaccard値を小さい順にソート
print(ans_jaccard_df.sort_values(by="jaccard値",ascending=True)[1:10])

           文書のペア  jaccard値
3    doc1 と doc5  0.101562
31   doc5 と doc7  0.101911
11   doc2 と doc5  0.107287
16  doc2 と doc10  0.109375
43  doc8 と doc10  0.109739
12   doc2 と doc6  0.110497
26   doc4 と doc7  0.115440
33   doc5 と doc9  0.118568
4    doc1 と doc6  0.119134


#### Dice

In [69]:
def dice_similarity(set_a, set_b):
    num_intersection =  len(set.intersection(set_a, set_b))
    sum_nums = len(set_a) + len(set_b)
    try:
        return 2 * num_intersection / sum_nums
    except ZeroDivisionError:
        return 1.0

In [70]:
cols = ['文書のペア', 'dice値']
ans_dice_df = pd.DataFrame(index=[], columns=cols)
#Jaccard係数を計算
for i in range(10):
    for j in range(10):
        if i < j:
            tmp_name = "doc" + str(i+1) + " と doc" +  str(j+1)
            record = pd.Series([tmp_name, dice_similarity(doc_set_list[i], doc_set_list[j])], index=ans_dice_df.columns)
            ans_dice_df = ans_dice_df.append(record, ignore_index=True)

In [71]:
ans_dice_df[1:5]

,文書のペア,dice値
1,doc1 と doc3,0.305810
2,doc1 と doc4,0.213953
3,doc1 と doc5,0.184397
4,doc1 と doc6,0.212903


In [72]:
# jaccard値を大きい順にソート
print(ans_dice_df.sort_values(by="dice値",ascending=False)[1:10])

          文書のペア     dice値
1   doc1 と doc3  0.305810
30  doc5 と doc6  0.297794
25  doc4 と doc6  0.291200
42  doc8 と doc9  0.286201
40  doc7 と doc9  0.278409
0   doc1 と doc2  0.272873
17  doc3 と doc4  0.270106
39  doc7 と doc8  0.269576
24  doc4 と doc5  0.267135


In [73]:
# jaccard値を大きい順にソート
print(ans_dice_df.sort_values(by="dice値",ascending=True)[1:10])

           文書のペア     dice値
3    doc1 と doc5  0.184397
31   doc5 と doc7  0.184971
11   doc2 と doc5  0.193784
16  doc2 と doc10  0.197183
43  doc8 と doc10  0.197775
12   doc2 と doc6  0.199005
26   doc4 と doc7  0.206986
33   doc5 と doc9  0.212000
4    doc1 と doc6  0.212903


#### Szymkiewicz-Simpson係数

In [74]:
def simpson_similarity(list_a, list_b):
    num_intersection = len(set.intersection(set(list_a), set(list_b)))
    min_num = min(len(set(list_a)), len(set(list_b)))
    try:
        return num_intersection / min_num
    except ZeroDivisionError:
        if num_intersection == 0:
            return 1.0
        else:
            return 0

In [75]:
cols = ['文書のペア', 'Szymkiewicz-Simpson']
ans_ss_df = pd.DataFrame(index=[], columns=cols)
#Jaccard係数を計算
for i in range(10):
    for j in range(10):
        if i < j:
            tmp_name = "doc" + str(i+1) + " と doc" +  str(j+1)
            record = pd.Series([tmp_name, simpson_similarity(doc_set_list[i], doc_set_list[j])], index=ans_ss_df.columns)
            ans_ss_df = ans_ss_df.append(record, ignore_index=True)

In [76]:
ans_ss_df[0:5]

,文書のペア,Szymkiewicz-Simpson
0,doc1 と doc2,0.280528
1,doc1 と doc3,0.312500
2,doc1 と doc4,0.215625
3,doc1 と doc5,0.213115
4,doc1 と doc6,0.220000


In [77]:
# jaccard値を大きい順にソート
print(ans_ss_df.sort_values(by="Szymkiewicz-Simpson",ascending=False)[1:10])

           文書のペア  Szymkiewicz-Simpson
9    doc2 と doc3             0.366337
30   doc5 と doc6             0.331967
42   doc8 と doc9             0.328125
39   doc7 と doc8             0.317221
29  doc4 と doc10             0.316923
1    doc1 と doc3             0.312500
24   doc4 と doc5             0.311475
44  doc9 と doc10             0.308594
25   doc4 と doc6             0.303333


In [78]:
# jaccard値を小さい順にソート
print(ans_ss_df.sort_values(by="Szymkiewicz-Simpson",ascending=True)[1:10])

           文書のペア  Szymkiewicz-Simpson
3    doc1 と doc5             0.213115
2    doc1 と doc4             0.215625
11   doc2 と doc5             0.217213
33   doc5 と doc9             0.217213
6    doc1 と doc8             0.218750
4    doc1 と doc6             0.220000
19   doc3 と doc6             0.226667
41  doc7 と doc10             0.232143
14   doc2 と doc8             0.234323


## ベクトルベース

In [157]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# 形態素解析した文書をTF-IDFを用いてベクトルに変換
doc_token_edit = []
for doc in doc_token:
    doc_token_edit.append('　'.join(doc))

vectorizer = CountVectorizer(token_pattern=u'(?u)\\b\\w+\\b')
transformer = TfidfTransformer()

tf = vectorizer.fit_transform(doc_token_edit) # 単語の出現頻度を計算
tfidf = transformer.fit_transform(tf) # 各ドキュメントのtfidfを計算
# print(tfidf)

### ベクトルベースの類似度
・コサイン類似度  

#### コサイン類似度

In [165]:
from sklearn.metrics.pairwise import cosine_similarity

# 類似度行列の作成
sim = cosine_similarity(tfidf)
# df作成
cols = ['sim[from][to]', 'cos類似度']
cos_df = pd.DataFrame(index=[], columns=cols)
for from_id in range(len(doc_list)):
    for to_id in range(from_id,len(doc_list)):
        if from_id != to_id:
            tmp_name = 'sim[{0}][{1}]'.format(from_id+1, to_id+1)
            record = pd.Series([tmp_name, sim[from_id][to_id]], index=cos_df.columns)
            cos_df = cos_df.append(record, ignore_index=True)

In [164]:
cos_df

,sim[from][to],cos類似度
0,sim[1][2],0.797852
1,sim[1][3],0.815287
2,sim[1][4],0.638131
3,sim[1][5],0.596893
4,sim[1][6],0.520321
5,sim[1][7],0.697017
6,sim[1][8],0.618456
7,sim[1][9],0.620049
8,sim[1][10],0.678041
9,sim[2][3],0.846432


In [166]:
#  cos類似度を大きい順にソート
print(cos_df.sort_values(by="cos類似度",ascending=False)[1:10])

   sim[from][to]    cos類似度
1      sim[1][3]  0.815287
0      sim[1][2]  0.797852
13     sim[2][7]  0.748806
39     sim[7][8]  0.744096
41    sim[7][10]  0.737126
42     sim[8][9]  0.719918
16    sim[2][10]  0.719403
5      sim[1][7]  0.697017
40     sim[7][9]  0.690478
